In [46]:
from transformers.models.llama import LlamaTokenizer
from transformers import LlamaForCausalLM
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import logging
import json

In [47]:
model_path = 'TinyLlama/TinyLlama-1.1B-step-50K-105b'
# model_path = 'tyzhu/tiny_LLaMA_1b_8k_cc_merged_v1_8k_iter-600000-ckpt-step-75000_hf'
model = AutoModelForCausalLM.from_pretrained(model_path, token = json.load(open('/home/aiops/zhuty/hf_token.json')))
# tokenizer = LlamaTokenizer.from_pretrained('/home/aiops/zhuty/tinyllama/models' , padding_side='left', truncation_side="left")
# tokenizer = AutoTokenizer.from_pretrained(model_path, token=json.load(open('/home/aiops/zhuty/hf_token.json')), use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
model.eval()
model = model.cuda()

In [48]:
prompt = "Hi my name is"
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False)

In [49]:
inputs = {k: v.cuda() for k, v in inputs.items()}

In [50]:
inputs

{'input_ids': tensor([[6324,  590, 1024,  338]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1]], device='cuda:0')}

In [51]:
result = model.generate(**inputs, max_new_tokens=3, do_sample=False, pad_token_id=tokenizer.eos_token_id)

In [52]:
tokenizer.decode(result[0])

'Hi my name is and I am'

In [10]:
tokenizer

LlamaTokenizerFast(name_or_path='tyzhu/tiny_LLaMA_1b_8k_cc_merged_v1_8k_iter-600000-ckpt-step-75000_hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='left', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [20]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model=model_path , token=json.load(open('/home/aiops/zhuty/hf_token.json')), device=0)

In [24]:
pipe.tokenizer(prompt)

{'input_ids': [1, 6324, 590, 1024, 338], 'attention_mask': [1, 1, 1, 1, 1]}

In [26]:
pipe.torch_dtype

torch.float32

In [29]:
pipe(prompt, max_new_tokens=30, do_sample=False)

[{'generated_text': 'Hi my name is Kyle. I\'m a 20 year old male. I\'m 5\'10" and 170 pounds.'}]